In [1]:
import dlc_bci as bci
import numpy as np

import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt

#pgdc de 316 et 100 c'est 4
mini_batch_size = 4

In [2]:
train_input, train_target  = bci.load(root = './data_bci')

print(str(type(train_input)),train_input.size())
print(str(type(train_target)),train_target.size())


test_input, test_target  = bci.load(root = './data_bci', train=False)

print(str(type(test_input)),test_input.size())
print(str(type(test_target)),test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


## Center, reduce and finalize format

In [3]:
train_moy = train_input.mean(dim=2).expand(train_input.size(2),train_input.size(0),train_input.size(1)).transpose(0,1).transpose(1,2)
train_std = train_input.std(dim=2).expand(train_input.size(2),train_input.size(0),train_input.size(1)).transpose(0,1).transpose(1,2)
train_input = train_input.sub_(train_moy).div(train_std)

test_moy = test_input.mean(dim=2).expand(test_input.size(2),test_input.size(0),test_input.size(1)).transpose(0,1).transpose(1,2)
test_std = test_input.std(dim=2).expand(test_input.size(2),test_input.size(0),test_input.size(1)).transpose(0,1).transpose(1,2)
test_input = test_input.sub_(test_moy).div(test_std)

In [4]:
train_input = train_input.view(len(train_input),1,train_input.size(1),-1)
test_input = test_input.view(len(test_input),1,test_input.size(1),-1)

train_input, test_input, train_target, test_target = Variable(train_input), Variable(test_input), Variable(train_target), Variable(test_target)

In [5]:
class second_model(nn.Module):
    def __init__(self, nb_hidden):
        super(second_model, self).__init__()
        self.conv1 = nn.Conv2d(1,32, kernel_size = 3)
        self.conv2 = nn.Conv2d(32,64, kernel_size = 3) 
        self.conv3 = nn.Conv2d(64,128, kernel_size = 3) 
        self.fc1 = nn.Linear(384, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)
        self.BN =  nn.BatchNorm1d(nb_hidden)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 2)) #4x32x13x24
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 2)) #4x64x5x11
        x = F.relu(F.max_pool2d(self.conv3(x), kernel_size = 3)) #4x128x1x3
        x = F.relu(self.fc1(x.view(mini_batch_size,-1)))
        x = self.BN(x)
        x = self.fc2(x)
        return x

In [6]:
# data already 1400 dimensions, implement a Convolution layer

class first_model(nn.Module):
    def __init__(self,nb_hidden):
        super(first_model, self).__init__()
        self.conv1 = nn.Conv2d(1,10, kernel_size = 7) #3x22x44
        self.fc1 = nn.Linear(1210, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)
        self.BN =  nn.BatchNorm1d(nb_hidden)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = (2,4))) # 3x11x11
        x = F.relu(self.fc1(x.view(mini_batch_size,-1)))
        x = self.BN(x)
        x = self.fc2(x)
        return x

In [7]:
def train_model(model, train_input, train_target,mini_batch_size):
    
    criterion = nn.CrossEntropyLoss()
    eta = 2e-2
    optimizer = torch.optim.SGD(model.parameters(),lr=eta,momentum=0.3)
    nb_epochs = 100

    for e in range(0, nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model.forward(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()

def compute_nb_errors(model, data_input, data_target, mini_batch_size):
    
    nb_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model.forward(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.data.max(1)
        for k in range(0, mini_batch_size):
            if (data_target.data[b + k] - predicted_classes[k] != 0):
                nb_errors = nb_errors + 1
    return int(nb_errors)

In [8]:

for nb in [10,50,100,200,500,1000]:
    model = second_model(nb)
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    nb_train_errors = compute_nb_errors(model, train_input, train_target, mini_batch_size)

    print('Second model : test error rate = {:.02f}% and train error rate = {:.02f}% with {} hidden units'
                  .format(nb_test_errors/test_input.size(0)*100,
                          nb_train_errors/train_input.size(0)*100,
                          nb))

Second model : test error rate = 50.00% and train error rate = 9.49% with 10 hidden units
Second model : test error rate = 47.00% and train error rate = 8.54% with 50 hidden units
Second model : test error rate = 41.00% and train error rate = 8.54% with 100 hidden units
Second model : test error rate = 50.00% and train error rate = 7.59% with 200 hidden units
Second model : test error rate = 49.00% and train error rate = 9.49% with 500 hidden units
Second model : test error rate = 49.00% and train error rate = 9.49% with 1000 hidden units
